In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("spark colab").getOrCreate()

In [3]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [4]:
key_path = key_path
project = project_id
dataset = 'bronze'
dataset_to_save = 'silver'
table = 'bronze_currency_data'
table_to_save = 'silver_currency_data'
table_conca = f'{project}.{dataset}.{table}'

In [5]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [6]:
df = bigquery.run_query(
    f"""
    SELECT
        *
    FROM {table_conca}
    """
)

In [7]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Currency
0,2015-01-01 00:00:00+00:00,1.209863,1.209863,1.209863,1.209863,1.209863,0,Euro (EURUSD)
1,2015-01-02 00:00:00+00:00,1.208868,1.208956,1.201080,1.208941,1.208941,0,Euro (EURUSD)
2,2015-01-05 00:00:00+00:00,1.195500,1.197590,1.188909,1.194643,1.194643,0,Euro (EURUSD)
3,2015-01-06 00:00:00+00:00,1.193830,1.197000,1.188693,1.193902,1.193902,0,Euro (EURUSD)
4,2015-01-07 00:00:00+00:00,1.187479,1.190000,1.180401,1.187536,1.187536,0,Euro (EURUSD)


In [8]:
# Creamos un df para Spark
spark_df = spark.createDataFrame(df)

In [12]:
from pyspark.sql.functions import first

df_pivot = spark_df.groupBy("Date").pivot("Currency").agg(first("Close"))

Py4JJavaError: An error occurred while calling o69.pivot.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 14 in stage 3.0 failed 1 times, most recent failure: Lost task 14.0 in stage 3.0 (TID 62) (CARLES-PC executor driver): java.net.SocketException: Connection reset
	at java.base/sun.nio.ch.NioSocketImpl.implRead(NioSocketImpl.java:318)
	at java.base/sun.nio.ch.NioSocketImpl.read(NioSocketImpl.java:346)
	at java.base/sun.nio.ch.NioSocketImpl$1.read(NioSocketImpl.java:796)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:1099)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:291)
	at java.base/java.io.BufferedInputStream.read1(BufferedInputStream.java:347)
	at java.base/java.io.BufferedInputStream.implRead(BufferedInputStream.java:420)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:399)
	at java.base/java.io.DataInputStream.readFully(DataInputStream.java:208)
	at java.base/java.io.DataIn

In [ ]:
df_renamed = df_pivot \
    .withColumnRenamed("Euro (EURUSD)", "EURUSD") \
    .withColumnRenamed("Libra Esterlina (GBPUSD)", "GBPUSD") \
    .withColumnRenamed("Yuan Chino (USDCNY)", "USDCNY") \
    .withColumnRenamed("Yen Japonés (JPYUSD)", "JPYUSD") \
    .withColumnRenamed("Dólar Australiano (AUDUSD)", "AUDUSD") \
    .withColumnRenamed("Franco Suizo (USDCHF)", "USDCHF")


In [ ]:
df_selected = df_renamed.select("Date", "EURUSD", "GBPUSD", "USDCNY", "JPYUSD", "AUDUSD", "USDCHF")
df_selected.show()

+-------------------+------------------+------------------+------------------+--------------------+------------------+------------------+
|               Date|            EURUSD|            GBPUSD|            USDCNY|              JPYUSD|            AUDUSD|            USDCHF|
+-------------------+------------------+------------------+------------------+--------------------+------------------+------------------+
|2019-06-18 00:00:00|1.1224603652954102|1.2541544437408447| 6.924900054931641|0.009213278070092201|0.6854009628295898|0.9986500144004822|
|2023-03-20 00:00:00|1.0678855180740356|1.2187690734863281| 6.886099815368652|0.007559493184089661|0.6713212132453918|0.9267399907112122|
|2024-07-31 00:00:00|1.0816072225570679|1.2836971282958984| 7.250699996948242|0.006550075486302376|0.6541501879692078|0.8820300102233887|
|2016-03-11 00:00:00|1.1178430318832397|1.4287755489349365| 6.498600006103516|0.008860534988343716|0.7462129592895508| 0.984499990940094|
|2022-01-03 00:00:00|1.13734591007

In [ ]:
# Convertir el DataFrame de PySpark a pandas
df_pandas = df_selected.toPandas()

In [ ]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df_pandas, project, dataset_to_save, table_to_save, if_exists='append', schema=None)

100%|██████████| 1/1 [00:00<00:00, 7612.17it/s]


In [ ]:
spark.stop()